# Extract a PowerPoint Presentation

**The old way:** Open PowerPoint, manually copy text from each slide, export slides as images one at a time, send the whole file to share just one slide.

**The new way:** Upload a `.pptx` to Istari, run extraction, get individual slides as PPTX/PDF/PNG, slide text as JSON, and the full deck as PDF and ODP automatically.

This notebook:
1. Navigates the Istari system hierarchy to find the PowerPoint file
2. Runs PowerPoint data extraction using Microsoft PowerPoint 2021 on Windows Server 2022
3. Views the 35 extracted artifacts (per-slide PPTX, PDF, PNG, text JSON; full deck PDF/ODP)
4. Snapshots the results so the team can see what was extracted

See [`example-output/`](example-output/) for pre-computed results.

In [ ]:
# Setup
import sys, json
from pathlib import Path

try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Explore the system — navigate the Istari hierarchy
#
# This system tracks one PowerPoint presentation (PPT_500KB_PPTX.pptx).
#
# Istari links:
#   System: https://demo.istari.app/systems/ff748bc2-7a66-4102-93f9-d5a554da6c1a

SYSTEM_ID = "ff748bc2-7a66-4102-93f9-d5a554da6c1a"  # Example: Extract PowerPoint
CONFIG_ID = "29e6030a-3360-48ee-b568-f03e288ae898"  # Baseline configuration

system = client.get_system(SYSTEM_ID)
print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
print(f"Configurations ({configs.total}):\n")

for config in configs.items:
    print(f"  {config.name}")
    print(f"    Config ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"    Tracked files ({tracked.total}):")
    for tf in tracked.items:
        mode = tf.specifier_type.value
        print(f"      {tf.file_id} ({mode})")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=10)
    print(f"    Snapshots ({snapshots.total}):")
    for snap in snapshots.items:
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = f"  [{', '.join(tag_names)}]" if tag_names else ""
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
        print(f"      {snap.id[:8]}...{tag_str}  ({revs.total} file(s))")
        for r in revs.items:
            size_kb = r.size / 1024
            print(f"        - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# Load the model — show file details and revision history

MODEL_ID = "82d6a6e9-7922-4226-a846-087200eb4a5d"  # PPT_500KB_PPTX.pptx

model = client.get_model(MODEL_ID)
name = model.display_name or model.file.revisions[0].name
print(f"Model: {name}")
print(f"  File ID:   {model.file.id}")
print(f"  Revisions: {len(model.file.revisions)}")
for rev in model.file.revisions:
    size_kb = rev.size / 1024
    ver = f"  ({rev.version_name})" if hasattr(rev, 'version_name') and rev.version_name else ""
    print(f"    {rev.name} — {size_kb:.1f} KB{ver}")
print(f"  Artifacts: {len(model.artifacts)}")
print(f"\nTracked in config: {CONFIG_ID[:8]}... (LATEST)")

In [ ]:
# Run PowerPoint extraction
#
# Sends the .pptx to an Istari agent running Microsoft PowerPoint.
# Extracts: individual slides as PPTX/PDF/PNG, slide text as JSON, full deck as PDF/ODP.
#
# Docs: https://docs.istaridigital.com/integrations/documents/microsoft_office_powerpoint

from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Submitting PowerPoint extraction job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@istari:extract",
    tool_name="microsoft_office_powerpoint",
    tool_version="2021",
    operating_system="Windows Server 2022",
    parameters={},
)
print(f"Job: {job.id}")

while True:
    sleep(10)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nExtraction complete!")
else:
    print("\nExtraction failed!")

In [ ]:
# View extraction results
#
# The agent produced 35 artifacts (8-slide deck):
#   whole_deck.pptx, whole_deck.pdf, whole_deck.odp
#   slide_N.pptx, slide_N.pdf, slide_N.png, slide_N_text.json  (for N = 1..8)

model = client.get_model(MODEL_ID)
print(f"Artifacts: {len(model.artifacts)}\n")

for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev:
        size_kb = rev.size / 1024
        ext = rev.name.split('.')[-1].lower() if '.' in rev.name else ''
        print(f"  {rev.name} ({size_kb:.1f} KB)")

        # Show JSON contents inline (500 char limit)
        if ext == 'json':
            try:
                data = json.loads(a.read_text())
                dumped = json.dumps(data, indent=2)
                print(f"    {dumped[:500]}")
                if len(dumped) > 500:
                    print("    ...")
            except Exception:
                pass
            print()

In [ ]:
# Snapshot: capture state after extraction

from istari_digital_client import NewSnapshot, NewSnapshotTag

print("Creating snapshot: post-extraction...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance

if hasattr(snapshot, "id"):
    snap_id = snapshot.id
    print(f"  New snapshot: {snap_id[:8]}...")
else:
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    snap_id = snaps.items[0].id
    print(f"  No changes — tagging existing snapshot: {snap_id[:8]}...")

client.create_tag(snap_id, NewSnapshotTag(tag="post-extraction"))
print("  Tagged: post-extraction")

revs = client.list_snapshot_revisions(snap_id, page=1, size=50)
print(f"  Files: {revs.total}")

In [ ]:
# View system state — all snapshots with tags, files, and sizes

print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
for config in configs.items:
    print(f"Configuration: {config.name}")
    print(f"  ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"  Tracked files: {tracked.total}")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=20)
    print(f"\n  Snapshots ({snapshots.total}):\n")

    for i, snap in enumerate(snapshots.items):
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = ", ".join(tag_names) if tag_names else "untagged"
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)

        print(f"    {i+1}. [{tag_str}]  ({revs.total} files)")
        for r in revs.items:
            size_kb = r.size / 1024
            if size_kb > 1024:
                print(f"       {r.name} ({size_kb/1024:.1f} MB)")
            else:
                print(f"       {r.name} ({size_kb:.1f} KB)")
        print()

## Summary

**What we did:**
1. Uploaded `PPT_500KB_PPTX.pptx` to Istari
2. Ran automated extraction — individual slides as PPTX/PDF/PNG, slide text as JSON, full deck as PDF and ODP
3. Viewed all 35 artifacts without opening PowerPoint
4. Snapshotted the extraction results for the team

### Version History

| # | Snapshot Tag | Files | What happened |
|---|-------------|-------|---------------|
| 1 | `initial-upload` | 1 | Raw PPT_500KB_PPTX.pptx uploaded |
| 2 | `post-extraction` | 36 | Original .pptx + 35 extracted artifacts |

### What Istari Did

| Step | Inner Loop (Microsoft PowerPoint 2021 on Windows Server 2022) | Outer Loop (Istari) |
|------|--------------------------------------------------------------|---------------------|
| Upload presentation | — | Stored .pptx, tracked as LATEST |
| Extract data | Opened deck in PowerPoint, split into 8 individual slides (PPTX + PDF + PNG each), exported per-slide text as JSON, rendered full deck as PDF and ODP | Ran job on Windows agent, stored all 35 artifacts |
| Snapshot | — | Captured point-in-time state with all 36 files |

### What's Next?

- **Share a single slide** — use `slide_N.pdf` or `slide_N.png` to share one slide without sending the full deck
- **Search slide text** — load all `slide_N_text.json` files to find keywords across the entire presentation programmatically
- **Compare revisions** — upload an updated .pptx, re-extract, and diff slide text JSON across versions